This code is taken from [here](https://blog.paperspace.com/seq-to-seq-attention-mechanism-keras/) and is modified to solve our task!

In [ ]:
# Please note that this code just follows the provided video
# Mount data from drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Folder name
folderName = 'CS685 Project/Sanity Check'
assert folderName is not None, "[Error] Please enter folder name."


# Load python files from our folder
import sys
sys.path.append('/content/drive/My Drive/{}'.format(folderName))

%cd /content/drive/My\ Drive/$folderName/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/176_V3hUNBvwdqw9WeYJjHSQNn8fY7Q2K/CS685 Project/Sanity Check


In [ ]:
!pip install sentencepiece
!pip install transformers
# !pip install torch
!pip install rich[jupyter]

# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console

     |████████████████████████████████| 1.2 MB 5.1 MB/s 
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 53.9 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
     |████████████████████████████████| 596 kB 58.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 231 kB 5.1 MB/s 
     |████████████████████████████████| 51 kB 4.1 MB/s 


Store Paraphrases as X and Shakespeare sentences as Z

Resulting dataset: (X, Z)

Task for dataset: Given Paraphrase X generate original sentence Z


Fine-tune T5 for this task

In [ ]:
# define a rich console logger
console = Console(record=True)

# to display dataframe in ASCII format
def display_df(df):
    """display dataframe in ASCII format"""

    console = Console()
    table = Table(
        Column("source_text", justify="center"),
        Column("target_text", justify="center"),
        title="Sample Data",
        pad_edge=False,
        box=box.ASCII,
    )

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    console.print(table)

# training logger to log training progress
training_logger = Table(
    Column("Epoch", justify="center"),
    Column("Steps", justify="center"),
    Column("Loss", justify="center"),
    title="Training Status",
    pad_edge=False,
    box=box.ASCII,
)

# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'


In [ ]:
class CustomDataSetClass(Dataset):
    """
    Creating a custom dataset for reading the dataset and
    loading it into the dataloader to pass it to the
    neural network for finetuning the model

    """

    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
    ):
        """
        Initializes a Dataset class

        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            source_len (int): Max length of source text
            target_len (int): Max length of target text
            source_text (str): column name of source text
            target_text (str): column name of target text
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]

    def __len__(self):
        """returns the length of dataframe"""

        return len(self.target_text)

    def __getitem__(self, index):
        """return the input ids, attention masks and target ids"""

        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])

        # cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())
        target_text = " ".join(target_text.split())

        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.summ_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }

In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.train()
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]

        if _ % 330 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss))
            console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [ ]:
def T5Trainer(
    dataframe, source_text, target_text, model_params, output_dir="./outputs/"
):

    """
    T5 trainer

    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
    model = model.to(device)

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]
    display_df(dataframe.head(2))

    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest for validation.
    train_size = 0.8
    train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = CustomDataSetClass(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = CustomDataSetClass(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    )

    # Training loop
    console.log(f"[Initiating Fine Tuning]...\n")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

    console.log(f"[Saving Model]...\n")
    # Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    # evaluating test dataset
    console.log(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
        final_df.to_csv(os.path.join(output_dir, "predictions.csv"))

    console.save_text(os.path.join(output_dir, "logs.txt"))

    console.log(f"[Validation Completed.]\n")
    console.print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n"""
    )
    console.print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [ ]:
# let's define model parameters specific to T5
model_params = {
    "MODEL": "t5-base",  # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE": 8,  # training batch size
    "VALID_BATCH_SIZE": 8,  # validation batch size
    "TRAIN_EPOCHS": 5,  # number of training epochs
    "VAL_EPOCHS": 1,  # number of validation epochs
    "LEARNING_RATE": 3e-4,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 255,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 255,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}

In [ ]:
path = "data/pseudo_parallel.csv"
df = pd.read_csv(path)
training_df = df.sample(frac = 0.9)
testing_df = df.drop(training_df.index, axis=0)
testing_df.size #shape (1839, 3)


5517

In [ ]:
# T5 accepts prefix of the task to be performed:
# Since we are summarizing, let's add summarize to source text as a prefix
model_df = training_df.copy()
# model_df["input"] = "Shakespeare it: " + model_df["input"]


T5Trainer(
    dataframe=model_df,
    source_text="input",
    target_text="target",
    model_params=model_params,
    output_dir="pseudo_parallel_removed_shakespeare",
)

[19:04:58] [Model]: Loading t5-base...                      <ipython-input-7-60af46234dab>:16
                                                                                             

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:169: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


[19:05:12] [Data]: Reading data...                          <ipython-input-7-60af46234dab>:27
                                                                                             

                                      Sample Data                                      
+-------------------------------------------------------------------------------------+
|            source_text             |                   target_text                  |
|------------------------------------+------------------------------------------------|
|How long shall her sweet life last? | But how long fairly shall her sweet life last ?|
|          There is peace.           |                     Peace !                    |
+-------------------------------------------------------------------------------------+

FULL Dataset: (16556, 2)

...

Completed 410

[20:29:21] [Validation Completed.]                         <ipython-input-7-60af46234dab>:105
                                                                                             

[Model] Model saved @ pseudo_parallel_removed_shakespeare/model_files

[Validation] Generation on Validation data saved @ 
pseudo_parallel_removed_shakespeare/predictions.csv

[Logs] Logs saved @ pseudo_parallel_removed_shakespeare/logs.txt

In [ ]:
def T5Generator(
    dataframe, source_text, target_text, model_params, input_dir="./outputs/", output_dir="./outputs/"
):

    """
    T5 trainer

    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # Loading the model
    path = os.path.join(input_dir, "model_files")

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained(path)

    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained(path)
    model = model.to(device)

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]
    display_df(dataframe.head(2))

    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest for validation.
    # train_size = 0.8
    # train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    gen_dataset = dataframe.reset_index(drop=True)
    # train_dataset = train_dataset.reset_index(drop=True)

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"Gen Dataset: {gen_dataset.shape}\n")

    gen_set = CustomDataSetClass(
        gen_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    gem_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    gen_loader = DataLoader(gen_set, **gem_params)

    # evaluating test dataset
    console.log(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, gen_loader)
        final_df = pd.DataFrame({"Generated Text": predictions})
        final_df.to_csv(os.path.join(output_dir, "gen_predictions.csv"))

    console.save_text(os.path.join(output_dir, "gen_logs.txt"))

In [ ]:
# gen_df = testing_df.copy()
# gen_df["input"] = "Shakespear it: " + gen_df["input"]
# gen_df.head(1)

In [ ]:
# T5 accepts prefix of the task to be performed:
# Since we are summarizing, let's add summarize to source text as a prefix
gen_df = testing_df.copy()
# gen_df["input"] = "Shakespear it: " + gen_df["input"]

T5Generator(
    dataframe=gen_df,
    source_text="input",
    target_text="target",
    model_params=model_params,
    input_dir="pseudo_parallel_removed_shakespeare",
    output_dir="pseudo_parallel_removed_shakespeare",
)

           [Model]: Loading t5-base...                     <ipython-input-11-283f28e2abf0>:19
                                                                                             

[20:29:24] [Data]: Reading data...                         <ipython-input-11-283f28e2abf0>:30
                                                                                             

                                    Sample Data                                     
+----------------------------------------------------------------------------------+
|          source_text           |                   target_text                   |
|--------------------------------+-------------------------------------------------|
|I have a plan to speak to thee. | I have a mind to strike thee ere thou speak'st .|
|      He is well, Madame.       |               Madam , he's well .               |
+----------------------------------------------------------------------------------+

FULL Dataset: (1839, 2)

Gen Dataset: (1839, 2)

...

Completed 180

Completed 190

Completed 200

Completed 210

Completed 220

In [ ]:
path = "data/dev_1.csv"
testing_df = pd.read_csv(path)
testing_df.head(1)

gen_df = testing_df.copy()
gen_df["Shakespearean"] = gen_df["text"]
gen_df["English"] =  gen_df["text"]

T5Generator(
    dataframe=gen_df,
    source_text="English",
    target_text="Shakespearean",
    model_params=model_params,
    input_dir="pseudo_parallel_removed_shakespeare",
    output_dir="T5_2-5-epochs-test1-outputs-removed-shakespeare/001",
)

[21:15:06] [Model]: Loading t5-base...                      <ipython-input-9-283f28e2abf0>:19
                                                                                             

[21:15:10] [Data]: Reading data...                          <ipython-input-9-283f28e2abf0>:30
                                                                                             

                                         Sample Data                                         
+-------------------------------------------------------------------------------------------+
|                source_text                  |                 target_text                 |
|---------------------------------------------+---------------------------------------------|
|     Now , you lie there on the path .       |      Now , you lie there on the path .      |
|She said if she were interested in someone , | She said if she were interested in someone ,|
|  it would be someone who looked like me .   |   it would be someone who looked like me .  |
+-------------------------------------------------------------------------------------------+

FULL Dataset: (1218, 2)

Gen Dataset: (1218, 2)

...

Completed 110

Completed 120

Completed 130

Completed 140

Completed 150